In [ ]:
''' formalizes the process of creating these ENSP only files '''

In [8]:
import pandas as pd
import os

In [14]:
def initial_vcf_filter(readpath, writepath):
    '''keep only the lines that contain ENSP'''

    with open(readpath, 'r') as rp:
        with open(writepath, 'w') as wp:
            lines = rp.readlines()
            for line in lines:
                if 'ENSP' in line:
                    wp.write(line)
                
        wp.close()
    rp.close()
    
    return 

In [15]:
def filter_variant_features(readpath, writepath):
    '''filter out the variant features we're not picking up''' 

    deplorables = ['|IGR|', '|FIVE_PRIME_UTR|', '|THREE_PRIME_UTR|', '|FIVE_PRIME_FLANK|', 
                   '|THREE_PRIME_FLANK|', '|COULD_NOT_DETERMINE|', '|SILENT|'] # does cerebra keep silent? 
    keep_lines = 0
    not_keep_lines = 0

    with open(readpath, 'r') as rp:
        with open(writepath, 'w') as wp:
            lines = rp.readlines()
            for line in lines:
                if any(ele in line for ele in deplorables): # filter all deplorables
                #if '|IGR|' in line: # filter out intergenic regions
                #if 'foobar' in line: # dont filter out anything
                    not_keep_lines += 1
                else:
                    keep_lines += 1
                    wp.write(line)
                
        wp.close()
    rp.close()
    
    return

In [16]:
def grep_proper_tags(readpath, writepath):
    '''pull out and filter ENSP tags'''
    
    # pull out just the ENSP tags
    rp0 = readpath
    wp0 = writepath
    cmd0 = 'grep -E -o "ENSP.{0,20}" ' + rp0 + ' > '+ wp0
    os.system(cmd0)
    
    # and now get just the ENSP tags with '.p' in them
    rp1 = wp0
    wp1 = rp1.rstrip('.vcf') + '_filt.vcf'
    #cmd1 = "grep ':p.' " + rp1 + ' > ' + wp1 # do filter
    cmd1 = "grep 'ENSP' " + rp1 + ' > ' + wp1 # dont do filter
    os.system(cmd1)
    
    # and finally, keep only the ENSP trunk labels
    rp2 = wp1
    wp2 = rp2.rstrip('.vcf') + '_final.vcf'
    cmd2 = 'grep -E -o "ENSP.{0,11}" ' + rp2 + ' > ' + wp2
    os.system(cmd2)
    
    return

In [17]:
basepath = '../funco_bench_raw/'

for file in os.listdir(basepath):
    fullpath = basepath + file
    
    rp0 = fullpath
    wp0 = fullpath.rstrip('.vcf') + '_ensp.vcf'
    initial_vcf_filter(rp0, wp0)
    
    rp1 = wp0
    wp1 = rp1.rstrip('.vcf') + '_clean.vcf'
    filter_variant_features(rp1, wp1)
    
    rp2 = wp1
    wp2 = rp2.rstrip('.vcf') + '_only.vcf'
    grep_proper_tags(rp2, wp2)

In [18]:
! mv ../funco_bench_raw/*ensp* ../funco_bench_ensp_tags/